# San Francisco Rental Prices Dashboard

Compiled the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [7]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [8]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [9]:
# Import the CSVs to Pandas DataFrames
file_path = Path("./Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("./Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In [10]:
# Defining variables to use in the functions for plots
neighborhood_avg_stats = (sfo_data.groupby(['neighborhood'],as_index=False).mean())

top_neighborhood_sale_price = (
    neighborhood_avg_stats.nlargest(
        10,
        'sale_price_sqr_foot'
    ).reset_index().drop(columns='index')
)

In [11]:
# Define Panel Visualization Functions
def housing_units_per_year():
    # Calculating the mean number of housing units per year
    housing_data = (sfo_data[['housing_units']].groupby('year').mean())
    housing_data = housing_data.hvplot.bar(
        x='year',
        y='housing_units',
        title='Housing Units in San Francisco from 2010 to 2016',
        xlabel='Year',
        ylabel='Housing Units',
        ylim = (370000,387500)
    ).opts(yformatter="%.0f")
    
    return housing_data

def average_gross_rent():
    # Calculate the average gross rent
    average_rent_sqr_foot = (sfo_data[['gross_rent']].groupby('year').mean())
    gross_rent_plot = average_rent_sqr_foot.hvplot.line(
        x='year',
        y='gross_rent',
        title='Average Gross Rent in San Francisco',
        xlabel='Year',
        ylabel='Gross Rent'
    )
    
    return gross_rent_plot

def average_sales_price():
    # Calculate the average sale price per square foot
    average_sale_price_sqft = (sfo_data[['sale_price_sqr_foot']].groupby('year').mean())
    rent_sqt_plot = average_sale_price_sqft.hvplot.line(
        x='year',
        y='sale_price_sqr_foot',
        title='Average Sale Price per Square Foot in San Francisco',
        xlabel='Year',
        ylabel='Avg. Sale Price'
    )
    
    return rent_sqt_plot

def average_price_by_neighborhood():
    sf_data_new = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    sf_sales_df = sf_data_new.reset_index()
    sales_plot = sf_sales_df.hvplot.line(
             x="year",
             y="sale_price_sqr_foot",
             colormap="viridis",
             title="neighborhood",
        groupby = "neighborhood"
    )
    return sales_plot

def top_most_expensive_neighborhoods():
    top_10_most_expensive = (neighborhood_avg_stats.nlargest(10,'sale_price_sqr_foot').reset_index().drop(columns='index'))
    expensive_plot = top_10_most_expensive.hvplot.bar(
        x="neighborhood",
         y="sale_price_sqr_foot",
         colormap="viridis",
         title="Top 10 Expensive Neighborhoods in SFO",
    height=500,
    rot=90,
    xlabel="Neighborhood",
    ylabel="Avg. Sale Price per Square Foot"
)
    
    return expensive_plot

def parallel_coordinates():
    parallel_coordinates_plot = px.parallel_coordinates(
    top_neighborhood_sale_price,
    dimensions=['sale_price_sqr_foot','housing_units','gross_rent'],
    color='sale_price_sqr_foot',
    labels={
            'sale_price_sqr_foot': 'Sale Price per Sq.ft.',
            'housing_units': 'Housing Units',
            'gross_rent': 'Gross Rent',
    },
    )
    
    return parallel_coordinates_plot


def parallel_categories():
    parallel_categories_plot = px.parallel_categories(
    top_neighborhood_sale_price,
    dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'],
    color='sale_price_sqr_foot',
    labels={
            'neighborhood':'Neighborhood',
            'sale_price_sqr_foot': 'Sale Price per Sq.ft.',
            'housing_units': 'Housing Units',
            'gross_rent': 'Gross Rent',
    },
    )
    
    return parallel_categories_plot

def neighborhood_map():
    neighborhood_stats_avg = (sfo_data.groupby('neighborhood',as_index=False).mean())
    combined_df = pd.concat(
        [df_neighborhood_locations, neighborhood_stats_avg],
        axis="columns",
        join="inner"
    ).drop(columns='neighborhood')
    
    # Plot scatter mapbox
    mapbox_plot = px.scatter_mapbox(
        combined_df,
        lat='Lat',
        lon='Lon',
        size='sale_price_sqr_foot',
        color='gross_rent',
        zoom=10,
        title='Average Sale Price per Square Foot and Gross Rent in San Francisco',
    )
    
    return mapbox_plot

## Panel Dashboard

In [13]:
# Create panels to structure the layout of the dashboard
welcome_panel = pn.Column(
    neighborhood_map()
)

yearly_market_analysis = pn.Row(
    housing_units_per_year(), 
    average_gross_rent(), 
    average_sales_price()
)

neighborhood_analysis = pn.Column(
    'Top Ten Most Expensive Neighborhoods',
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_plot_analysis = pn.Column(
    '## Parallel Coordinates and Categories',
    parallel_coordinates(),
    parallel_categories()
)

# Create tabs
panel = pn.Tabs(
    ('Welcome', welcome_panel),
    ('Yearly Market Analysis', yearly_market_analysis),
    ('Neighborhood Analysis', neighborhood_analysis),
    ('Parallel Plot Analysis', parallel_plot_analysis)
)

## Serve the Panel Dashboard

In [15]:
panel.servable()

Tabs(active=3)
    [0] Column
        [0] Plotly(Figure, relayout_data={'autosize': True})
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True})
        [2] Plotly(Figure, relayout_data={'autosize': True})